In [ ]:
#Libraries
import pandas as pd
import numpy as np
import glob
import os

In [ ]:
#Set Variables and map to Folder
import_path = 'C:/Users/ianmc/OneDrive/Documents/Work/Data/news_blogs_coronavirus'
os.chdir(import_path)
all_filenames = [i for i in glob.glob('*.{}'.format('JSON'))]
print(all_filenames)

#Read in file(s)
json_data = pd.concat((pd.read_json(f,orient='columns', lines=True) for f in all_filenames), ignore_index=True)
#json_data = pd.read_json(all_filenames[0], lines=True)

In [ ]:
#Drop Unsused Columns to run faster
unused_columns = ['organizations', 'uuid', 'author', 'url', 'ord_in_thread','title', 'locations', 'entities',
                  'highlightText', 'language','persons', 'text', 'external_links', 'crawled','highlightTitle']
json_data.drop(unused_columns, axis=1)

#change date columns to show just date
json_data['published'] = pd.to_datetime(json_data['published']).dt.date
#json_data['crawled'] = pd.to_datetime(json_data['crawled']).dt.date #Not using this column

#filter to just US
#for decmeber which only had 1 thread per entry
us_json_data = json_data[pd.Series(json_data['thread'][rn]['country'] for rn in np.arange(json_data.shape[0])).str.contains('US', na=False)]

#add a column with a value of 1 to count the reports per a day
us_json_data.insert(column='count', loc=1, value=1)

#create a second table pivotiting on dates to count the reports per a day
us_count_per_day = us_json_data.pivot_table( values='count', index='published', aggfunc=np.sum)

In [ ]:
#Example plot for data
import matplotlib.pyplot as plt

x = us_count_per_day.reset_index()['published']
y = us_count_per_day.reset_index()['count']
labels = us_count_per_day.reset_index()['published']

plt.plot(x, y)
plt.xticks(x, labels, rotation='90')
plt.subplots_adjust(bottom=0.15)
plt.show()

In [ ]:
export_path = 'C:/Users/ianmc/OneDrive/Documents/Work/Data/news_blogs_coronavirus/IEEE_US_Counts_2020_02.csv'
us_count_per_day.to_csv((export_path), index = True, header = True, encoding='utf-8-sig')